# Import Libraries

In [ ]:
from time import sleep
import requests
import time
import selenium
from parsel import Selector
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import csv
import pandas as pd

# Functions to scrape various elements

In [2]:
def get_review(element):
    global review
    try:
        review = element.find_element_by_xpath("./div[3]/p[2]").text
    except selenium.common.exceptions.NoSuchElementException:
        return False
    return review
####################

def get_rating(element):
    global rating
    try:
        rating = element.find_element_by_xpath("./div[1]/div/img").get_attribute("data-rating")
        #print(rating)
    except selenium.common.exceptions.NoSuchElementException:
        return 0
    return rating
########################  

def get_custinfo(element):
    global cust_info
    try:
        cust_info = element.find_element_by_xpath("./div[2]/div[2]/strong[1]").text
        #print(rating)
    except selenium.common.exceptions.NoSuchElementException:
        return 'NA'
    return cust_info
#####################

def get_no_of_people(element):
    global no_people
    try:
        no_people = element.find_element_by_xpath("./div[4]/span/strong").text
        #print(rating)
    except selenium.common.exceptions.NoSuchElementException:
        return '0'
    return no_people
#####################

def get_date(element):
    global dte
    try:
        dte = element.find_element_by_xpath("./div[3]/span").text
        #print(rating)
    except selenium.common.exceptions.NoSuchElementException:
        return '0'
    return dte
#########################

def go_to_next_page():
    
    href = browser.find_element_by_xpath("//a[@class = 'ca-a-md ca-a-uprcs ca-a-blk prf-pgr__nxt js-pager-next']").get_attribute("href")
    #print('Going to {href}')
    browser.get(href)
#########################

def has_next_page():
    
    try:
        boo = browser.find_element_by_xpath("//a[@class = 'ca-a-md ca-a-uprcs ca-a-blk prf-pgr__nxt js-pager-next']")
        return True
    
    except selenium.common.exceptions.NoSuchElementException:
        return False
    

# Calling an instance of browser through Selenium and scraping the data automatically

In [3]:
def get_browser():
    chrome_options = wd.ChromeOptions()
    browser = wd.Chrome(executable_path='C:/Users/yogilla/Desktop/College_Course/Data Management Project/chromedriver_win32/chromedriver.exe',options=chrome_options)
    return browser
browser = get_browser()
######################

def main():
   
    main_list = []
    rating_list =[]
    cust_nameList=[]
    cust_stateList =[]
    people_noList=[]
    dateList = []
    browser.get("https://www.consumeraffairs.com/insurance/kaiser.html")        #directs to the desired site 
    
    flag = 1
    while(flag == 1):
        
        #print('im in')
        review_cards = browser.find_elements_by_xpath("//div[contains(@class,'js-rvw')]")  #this xpath is customized (see bottom of the file) 
        for i in range(len(review_cards)):
                #print(i)
            review = get_review(review_cards[i])
            rating = get_rating(review_cards[i])
            cust_info = get_custinfo(review_cards[i])
            no_people = get_no_of_people(review_cards[i])
            dte = get_date(review_cards[i])
            #print(cust_info)
            main_list.append([review])
            rating_list.append([rating])
            people_noList.append([no_people])
            
            if dte == '0':
                dateList.append('0')
            else:
                dateList.append(dte.split(': ')[1])
            if cust_info != 'NA':
                cust_split =cust_info.split('of') 
                cust_nameList.append(cust_split[0])
                cust_stateList.append(cust_split[1])
            else:
                cust_nameList.append('NAAA')
                cust_stateList.append('NAAA')
        if has_next_page():
            go_to_next_page()

        else:
            flag = 0

    d = {'Review':main_list,'Rating':rating_list,'Name':cust_nameList,'Address':cust_stateList, 'Num of People' : people_noList, 'Date':dateList}
    df = pd.DataFrame(d)
    
    df.to_csv("reviews1.csv",mode ='a',index = True,header = True)
    #print(df)    
    
if __name__ == '__main__':
    main()

